In [26]:
import geopandas as gpd
import json
import ssl 
import geemap
ssl._create_default_https_context = ssl._create_unverified_context
import ee

# Trigger the authentication flow.
# Don't need to run the below as I am alreay authenticated
# ee.Authenticate()

# Initialize the library.
ee.Initialize()

In [1]:
land_cover_dict = {
    "Tree cover":{
        "color": "#006400",
        "value": 10, 
    },
    "Shrubland":{
        "color": "#ffbb22",
        "value": 20, 
    },
    "Grassland":{
        "color": "#ffff4c",
        "value": 30, 
    },
    "Cropland":{
        "color": "#f096ff",
        "value": 40, 
    },
    "Built-up":{
        "color": "#fa0000",
        "value": 50, 
    },
    "Bare / sparse vegetation":{
        "color": "#b4b4b4",
        "value": 60, 
    },
    "Snow and ice":{
        "color": "#f0f0f0",
        "value": 70, 
    },
    "Permanent water bodies":{
        "color": "#0064c8",
        "value": 80, 
    },
    "Herbaceous wetland":{
        "color": "#0096a0",
        "value": 90, 
    },
    "Mangroves":{
        "color": "#00cf75",
        "value": 95, 
    },
    "Moss and lichen":{ 
        "color": "#fae6a0",
        "value": 100, 
    },
}



In [20]:
from typing import Dict, Optional
def ee_geo_compatible(geoJSON_input: Dict):
    geoJSON_output = dict(geoJSON_input)
    try:
        if geoJSON_input["type"]=="Polygon":
            check_coords = geoJSON_input["coordinates"][0][0]
            if len(check_coords)>2:
                print("Forcefully making coordinates compatible ee.Geometry as current len>2:", check_coords)
                new_coords = [[x[0:2] for x in y] for y in geoJSON_input["coordinates"]]
                geoJSON_output["coordinates"] = new_coords
        else:
            print("currently unknown handling for type:", geoJSON_input["type"])
        return geoJSON_output
    except:
        print("Dict passed does not follow geoJSON format")
        return None
            
    

In [21]:
import geopandas as gpd
# Path to your KML file
kml_file_path = 'townsville.kml'

# Read the KML file using geopandas
gpd.io.file.fiona.drvsupport.supported_drivers['KML'] = 'rw'
my_map = gpd.read_file(kml_file_path, driver='KML')
kml_geo_json=json.loads(my_map.to_json())
new_kml_geo_json = dict(kml_geo_json)
# Initiate with no features
new_kml_geo_json["features"] = []
display("original geojson")
display(kml_geo_json)
for feat in kml_geo_json["features"]:
    new_feat = dict(feat)
    new_feat["geometry"] = ee_geo_compatible(feat["geometry"])
    if new_feat["geometry"]:
        try:
            ee.Geometry(new_feat["geometry"])
            print("succesfully passed to ee.Geometry")
            new_kml_geo_json["features"] += [new_feat]
        except:
            print("Unable to pass into ee.Geometry asses below feature")
            print(new_feat)
            
display("updated geoJSON")
display(new_kml_geo_json)


'original geojson'

{'type': 'FeatureCollection',
 'features': [{'id': '0',
   'type': 'Feature',
   'properties': {'Name': 'townsville polygon', 'Description': ''},
   'geometry': {'type': 'Polygon',
    'coordinates': [[[146.6789497574668, -19.23247934166526, 0.0],
      [146.6879911595589, -19.33054650116621, 0.0],
      [146.8302295699071, -19.33087430904637, 0.0],
      [146.8595478481039, -19.26336834562065, 0.0],
      [146.8048945577382, -19.2333878909012, 0.0],
      [146.6789497574668, -19.23247934166526, 0.0]]]}}]}

[146.6789497574668, -19.23247934166526, 0.0]
Forcefully making coordinates compatible ee.Geometry as current len>2: [146.6789497574668, -19.23247934166526, 0.0]
[[[146.6789497574668, -19.23247934166526], [146.6879911595589, -19.33054650116621], [146.8302295699071, -19.33087430904637], [146.8595478481039, -19.26336834562065], [146.8048945577382, -19.2333878909012], [146.6789497574668, -19.23247934166526]]]
succesfully passed to ee.Geometry


'updated geoJSON'

{'type': 'FeatureCollection',
 'features': [{'id': '0',
   'type': 'Feature',
   'properties': {'Name': 'townsville polygon', 'Description': ''},
   'geometry': {'type': 'Polygon',
    'coordinates': [[[146.6789497574668, -19.23247934166526],
      [146.6879911595589, -19.33054650116621],
      [146.8302295699071, -19.33087430904637],
      [146.8595478481039, -19.26336834562065],
      [146.8048945577382, -19.2333878909012],
      [146.6789497574668, -19.23247934166526]]]}}]}

In [24]:
geojson_fc = ee.FeatureCollection(new_kml_geo_json)
display(geojson_fc.geometry())

ee.Geometry({
  "functionInvocationValue": {
    "functionName": "Collection.geometry",
    "arguments": {
      "collection": {
        "functionInvocationValue": {
          "functionName": "Collection",
          "arguments": {
            "features": {
              "arrayValue": {
                "values": [
                  {
                    "functionInvocationValue": {
                      "functionName": "Feature",
                      "arguments": {
                        "geometry": {
                          "functionInvocationValue": {
                            "functionName": "GeometryConstructors.Polygon",
                            "arguments": {
                              "coordinates": {
                                "constantValue": [
                                  [
                                    [
                                      146.6789497574668,
                                      -19.23247934166526
                                

In [62]:
dataset = ee.ImageCollection('ESA/WorldCover/v100').first()
display(dataset.getInfo())
visualization = {
    'bands': ['Map'],
}

# display(dataset.getInfo())

m = geemap.Map()
poly = geojson_fc.geometry()
m.add_ee_layer(dataset.clip(poly), visualization, 'Landcover')
m.add_legend(title="ESA Land Cover", builtin_legend='ESA_WorldCover')
m.centerObject(poly, 12)

display(m)


{'type': 'Image',
 'bands': [{'id': 'Map',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 255},
   'dimensions': [4320000, 1728000],
   'crs': 'EPSG:4326',
   'crs_transform': [8.333333333333333e-05,
    0,
    -180,
    0,
    -8.333333333333333e-05,
    84]}],
 'version': 1685065671563344,
 'id': 'ESA/WorldCover/v100/2020',
 'properties': {'system:time_start': 1577836800000,
  'Map_class_names': ['Tree cover',
   'Shrubland',
   'Grassland',
   'Cropland',
   'Built-up',
   'Bare / sparse vegetation',
   'Snow and ice',
   'Permanent water bodies',
   'Herbaceous wetland',
   'Mangroves',
   'Moss and lichen'],
  'system:footprint': {'type': 'LinearRing',
   'coordinates': [[-180, -90],
    [180, -90],
    [180, 90],
    [-180, 90],
    [-180, -90]]},
  'system:time_end': 1609459200000,
  'Map_class_palette': ['006400',
   'ffbb22',
   'ffff4c',
   'f096ff',
   'fa0000',
   'b4b4b4',
   'f0f0f0',
   '0064c8',
   '0096a0',
   '00cf75',
   'fae6a

Map(center=[-19.281796805191107, 146.76263396612705], controls=(WidgetControl(options=['position', 'transparen…

In [50]:
dataset = ee.ImageCollection('ESA/WorldCover/v200').first()

visualization = {
    'bands': ['Map'],
}

# display(dataset.getInfo())

m = geemap.Map()
poly = geojson_fc.geometry()
m.add_ee_layer(dataset.clip(poly), visualization, 'Landcover')
m.add_legend(title="ESA Land Cover", builtin_legend='ESA_WorldCover')

m.centerObject(poly, 12)

display(m)


Map(center=[-19.281796805191107, 146.76263396612705], controls=(WidgetControl(options=['position', 'transparen…

In [74]:
# These consistent Land Cover maps (v3.0.1) are provided for the period 2015-2019 over the entire Glo
dataset_2015 = ee.Image('COPERNICUS/Landcover/100m/Proba-V-C3/Global/2015').select(
    'discrete_classification'
)
dataset_2019 = ee.Image('COPERNICUS/Landcover/100m/Proba-V-C3/Global/2019').select(
    'discrete_classification'
)

visualization = {
    'bands': ['discrete_classification'],
}

# display(dataset.getInfo())

m = geemap.Map()
poly = geojson_fc.geometry()
m.add_ee_layer(dataset_2015.clip(poly), visualization, 'Landcover_2015')
m.add_ee_layer(dataset_2019.clip(poly), visualization, 'Landcover_2019')
m.add_legend(title="ESA Land Cover", builtin_legend='COPERNICUS/Landcover/100m/Proba-V/Global')
m.centerObject(poly, 12)

display(m)

Map(center=[-19.281796805191107, 146.76263396612705], controls=(WidgetControl(options=['position', 'transparen…

In [69]:
dataset = ee.Image('ESA/GLOBCOVER_L4_200901_200912_V2_3').select(
    'landcover'
)

visualization = {
    # 'palette': landcover.get("visualization_0_palette").getInfo().strip("'").split(","),
}

m = geemap.Map()
poly = geojson_fc.geometry()
m.add_ee_layer(dataset.clip(poly), visualization, 'Landcover')
m.add_legend(title="Landcover", builtin_legend='GLOBCOVER')
m.centerObject(poly, 12)

display(m)

Map(center=[-19.281796805191107, 146.76263396612705], controls=(WidgetControl(options=['position', 'transparen…

In [51]:
legends = geemap.builtin_legends
for legend in legends:
    print(legend)

NLCD
ESA_WorldCover
ESRI_LandCover
ESRI_LandCover_TS
Dynamic_World
NWI
MODIS/051/MCD12Q1
MODIS/006/MCD12Q1
GLOBCOVER
JAXA/PALSAR
Oxford
AAFC/ACI
COPERNICUS/CORINE/V20/100m
COPERNICUS/Landcover/100m/Proba-V/Global
USDA/NASS/CDL
ALOS_landforms


In [73]:
dataset_esa = ee.ImageCollection('ESA/WorldCover/v200').first()

visualization = {
    'bands': ['Map'],
}

# display(dataset.getInfo())

m = geemap.Map()
poly = geojson_fc.geometry()
m.add_ee_layer(dataset_esa.clip(poly), visualization, 'ESA_WorldCover')
m.add_legend('ESA_WorldCover', builtin_legend='ESA_WorldCover', layer_name='ESA_WorldCover')

dataset_glob = ee.Image('ESA/GLOBCOVER_L4_200901_200912_V2_3').select(
    'landcover'
)

visualization = {
    # 'palette': landcover.get("visualization_0_palette").getInfo().strip("'").split(","),
}

m.add_ee_layer(dataset_glob.clip(poly), visualization, 'GLOBCOVER')
m.add_legend('GLOBCOVER', builtin_legend='GLOBCOVER', layer_name='GLOBCOVER')
m.centerObject(poly, 12)

display(m)


Map(center=[-19.281796805191107, 146.76263396612705], controls=(WidgetControl(options=['position', 'transparen…

In [88]:
def mask_s2_clouds(image):
  """Masks clouds in a Sentinel-2 image using the QA band.

  Args:
      image (ee.Image): A Sentinel-2 image.

  Returns:
      ee.Image: A cloud-masked Sentinel-2 image.
  """
  qa = image.select('QA60')

  # Bits 10 and 11 are clouds and cirrus, respectively.
  cloud_bit_mask = 1 << 10
  cirrus_bit_mask = 1 << 11

  # Both flags should be set to zero, indicating clear conditions.
  mask = (
      qa.bitwiseAnd(cloud_bit_mask)
      .eq(0)
      .And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))
  )

  return image.updateMask(mask).divide(10000)

rgb_vis = {
    'min': 0.0,
    'max': 0.3,
    'bands': ['B4', 'B3', 'B2'],
}

m = geemap.Map()
m.centerObject(poly, 12)

for yr in ['2017','2019', '2021']:
        
    dataset = (
        ee.ImageCollection('COPERNICUS/S2_HARMONIZED')
        .filterDate(f'{yr}-01-01', f'{yr}-12-31')
        # Pre-filter to get less cloudy granules.
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))
        .map(mask_s2_clouds)
    )

    m.add_layer(dataset.median().clip(poly), rgb_vis, f'{yr}_Satalite')
# m.add_layer(dataset_2016.median(), rgb_vis, '2016_Satalite')
m

Map(center=[-19.281796805191107, 146.76263396612705], controls=(WidgetControl(options=['position', 'transparen…